In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import os
import matplotlib.pyplot as plt
import torch

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# the config of heatmap needs the original WSI

# Autenticazione “di servizio” fornita da Colab
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = '1gtE_lNw4MYiarzIu4qVAo-uu6aSzxS1d'
os.makedirs('/content/ndpi_files', exist_ok=True)

# Elenca e scarica tutti i file nella cartella
file_list = drive.ListFile(
    {'q': f"'{folder_id}' in parents and trashed=false"}
).GetList()

# 1) Definisci gli array
B = [
    "M-1.ndpi",
    '''
    "M-104.ndpi","M-105.ndpi","M-108.ndpi",
    "M-112.ndpi","M-121.ndpi","M-24.ndpi","M-30.ndpi",
    "M-32.ndpi","M-36.ndpi"
    '''
]
E = [
    "M-10.ndpi",
    '''
    "M-100.ndpi","M-102.ndpi","M-103.ndpi",
    "M-107.ndpi","M-109.ndpi","M-11.ndpi","M-110.ndpi",
    "M-111.ndpi","M-113.ndpi"
    '''
]
S = [
    "M-101.ndpi",
    '''
    "M-114.ndpi","M-65.ndpi","M-86.ndpi","M-87.ndpi"
    '''
]

base_dir = '/content/ndpi_files'
for sub in ('B','E','S'):
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

for f in file_list:
    title = f['title']
    if title in B:
        dest = os.path.join(base_dir, 'B', title)
    elif title in E:
        dest = os.path.join(base_dir, 'E', title)
    elif title in S:
        dest = os.path.join(base_dir, 'S', title)
    else:
        continue

    print(f"Scarico {title} in {os.path.dirname(dest)} …")
    f.GetContentFile(dest)

In [ ]:
# the config for the heatmap needs this csv file
# Combine arrays with their labels

data = []
for slide in B:
    data.append([slide.replace(".ndpi", ""), "B"])
for slide in E:
    data.append([slide.replace(".ndpi", ""), "E"])
for slide in S:
    data.append([slide.replace(".ndpi", ""), "S"])

# Specify the full path for the output file
output_file = r"MLIAProject\CLAM\heatmaps\process_lists\heatmap_dataset.csv"

# Write to CSV file
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["slide_id", "label"])  # Write header
    writer.writerows(data)  # Write data

print(f"CSV file '{output_file}' created successfully.")

In [ ]:
# the config of heatmap needs a checkpoint to initialize the model
# (in this case results/MLIA_CLAM_50_s1/s_0_checkpoint.pt)

!zip -r result_classifier.zip 'results/'

In [ ]:
# !MPLBACKEND=Agg CUDA_VISIBLE_DEVICES=0 conda run -n clam_latest python MLIAProject/CLAM/create_heatmaps.py --config MLIAProject/CLAM/heatmaps/configs/MLIAProject_config.yaml
